In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
#Linking drive to colab to store datasets
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library. Though the link asks you to verify twice, you don't have to!
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/

In [7]:
# Create a file in a new directory called "Kaggle" in your Google Drive. This will be your operation base :P
#!echo "This newly created file will appear in your Drive file list. If you are reading this, that means the attempt to integrate was successful" > drive/EIP/created.txt
!cp -r drive/EIP/* .
!ls

attempt_1			  weights-improvement-30-0.85.hdf5
datalab				  weights-improvement-30-0.86.hdf5
DNST_model.h5			  weights-improvement-31-0.87.hdf5
drive				  weights-improvement-33-0.86.hdf5
weights-improvement-25-0.84.hdf5  weights-improvement-39-0.87.hdf5
weights-improvement-26-0.83.hdf5


In [0]:
#For architecture
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

#For loading models
from pathlib import Path
from keras.models import load_model

#For data augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.1

In [11]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 29s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 80
dropout_rate = 0.4
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (5,5), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [56]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_261 (Conv2D)             (None, 32, 32, 80)   6000        input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_261 (BatchN (None, 32, 32, 80)   320         conv2d_261[0][0]                 
__________________________________________________________________________________________________
activation_261 (Activation)     (None, 32, 32, 80)   0           batch_normalization_261[0][0]    
__________________________________________________________________________________________________
conv2d_262

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# checkpoint
from keras.callbacks import ModelCheckpoint
filepath= "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
#1-10
epoch = 0
e = 10
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 628s 13ms/step - loss: 1.5076 - acc: 0.4483 - val_loss: 1.7112 - val_acc: 0.4838

Epoch 00001: val_acc improved from -inf to 0.48380, saving model to weights-improvement-01-0.48.hdf5
Epoch 2/10
13952/50000 [=======>......................] - ETA: 6:54 - loss: 1.1552 - acc: 0.584350000/50000 [==============================] - 620s 12ms/step - loss: 1.0596 - acc: 0.6198 - val_loss: 1.9296 - val_acc: 0.5166

Epoch 00002: val_acc improved from 0.48380 to 0.51660, saving model to weights-improvement-02-0.52.hdf5
Epoch 3/10
29312/50000 [================>.............] - ETA: 3:57 - loss: 0.8666 - acc: 0.690250000/50000 [==============================] - 619s 12ms/step - loss: 0.8380 - acc: 0.7006 - val_loss: 1.0405 - val_acc: 0.6769

Epoch 00003: val_acc improved from 0.51660 to 0.67690, saving model to weights-improvement-03-0.68.hdf5
Epoch 4/10
33280/50000 [==================>........

MessageError: ignored

In [18]:
!ls
#!cp *.hdf5 drive/EIP/
#!cp *.h5 drive/EIP

attempt_1			  weights-improvement-30-0.85.hdf5
datalab				  weights-improvement-30-0.86.hdf5
DNST_model.h5			  weights-improvement-31-0.87.hdf5
drive				  weights-improvement-33-0.86.hdf5
weights-improvement-25-0.84.hdf5  weights-improvement-39-0.87.hdf5
weights-improvement-26-0.83.hdf5


In [0]:
model.load_weights('weights-improvement-09-0.77.hdf5')

#10-20
epoch = 10
e = 20

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Train on 50000 samples, validate on 10000 samples
Epoch 11/20
50000/50000 [==============================] - 669s 13ms/step - loss: 0.4284 - acc: 0.8510 - val_loss: 1.0082 - val_acc: 0.7369

Epoch 00011: val_acc improved from -inf to 0.73690, saving model to weights-improvement-11-0.74.hdf5
Epoch 12/20
13696/50000 [=======>......................] - ETA: 7:10 - loss: 0.3869 - acc: 0.865750000/50000 [==============================] - 639s 13ms/step - loss: 0.3957 - acc: 0.8625 - val_loss: 0.7689 - val_acc: 0.7897

Epoch 00012: val_acc improved from 0.73690 to 0.78970, saving model to weights-improvement-12-0.79.hdf5
Epoch 13/20
29184/50000 [================>.............] - ETA: 4:06 - loss: 0.3551 - acc: 0.874950000/50000 [==============================] - 639s 13ms/step - loss: 0.3599 - acc: 0.8740 - val_loss: 0.7729 - val_acc: 0.7882

Epoch 00013: val_acc did not improve from 0.78970
Epoch 14/20
39808/50000 [======================>.......] - ETA: 2:00 - loss: 0.3377 - acc: 0.882350000

In [0]:
!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

In [19]:
print('Using real-time data augmentation.')

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator( 
    rotation_range=90, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True) 

datagen.fit(x_train)

Using real-time data augmentation.


In [0]:
model.load_weights('weights-improvement-20-0.82.hdf5')

#20-25
epoch=20
e = 25
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

Train on 50000 samples, validate on 10000 samples
Epoch 21/25
50000/50000 [==============================] - 670s 13ms/step - loss: 0.2766 - acc: 0.9024 - val_loss: 0.7347 - val_acc: 0.8164

Epoch 00021: val_acc improved from -inf to 0.81640, saving model to weights-improvement-21-0.82.hdf5
Epoch 22/25
13696/50000 [=======>......................] - ETA: 7:09 - loss: 0.2579 - acc: 0.908150000/50000 [==============================] - 636s 13ms/step - loss: 0.2602 - acc: 0.9077 - val_loss: 0.7440 - val_acc: 0.8308

Epoch 00022: val_acc improved from 0.81640 to 0.83080, saving model to weights-improvement-22-0.83.hdf5
Epoch 23/25
29184/50000 [================>.............] - ETA: 4:05 - loss: 0.2437 - acc: 0.911950000/50000 [==============================] - 635s 13ms/step - loss: 0.2516 - acc: 0.9092 - val_loss: 0.7689 - val_acc: 0.8195

Epoch 00023: val_acc did not improve from 0.83080
Epoch 24/25
39808/50000 [======================>.......] - ETA: 2:00 - loss: 0.2347 - acc: 0.917650000

In [23]:
model.load_weights('weights-improvement-25-0.84.hdf5')
#25-35
epoch = 25
e = 35
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

Train on 50000 samples, validate on 10000 samples
Epoch 26/35
50000/50000 [==============================] - 654s 13ms/step - loss: 0.2298 - acc: 0.9188 - val_loss: 0.7404 - val_acc: 0.8284

Epoch 00026: val_acc improved from -inf to 0.82840, saving model to weights-improvement-26-0.83.hdf5
Epoch 27/35
13696/50000 [=======>......................] - ETA: 7:04 - loss: 0.2029 - acc: 0.9287

50000/50000 [==============================] - 629s 13ms/step - loss: 0.2153 - acc: 0.9237 - val_loss: 0.9706 - val_acc: 0.7929

Epoch 00027: val_acc did not improve from 0.82840
Epoch 28/35
34944/50000 [===================>..........] - ETA: 2:55 - loss: 0.2062 - acc: 0.9278

50000/50000 [==============================] - 629s 13ms/step - loss: 0.2103 - acc: 0.9261 - val_loss: 0.9227 - val_acc: 0.8015

Epoch 00028: val_acc did not improve from 0.82840
Epoch 29/35
41472/50000 [=======================>......] - ETA: 1:39 - loss: 0.1992 - acc: 0.9294

50000/50000 [==============================] - 629s 13ms/step - loss: 0.1998 - acc: 0.9287 - val_loss: 0.9477 - val_acc: 0.8048

Epoch 00029: val_acc did not improve from 0.82840
Epoch 30/35
43520/50000 [=========================>....] - ETA: 1:15 - loss: 0.1961 - acc: 0.9304

50000/50000 [==============================] - 628s 13ms/step - loss: 0.1955 - acc: 0.9305 - val_loss: 0.6155 - val_acc: 0.8567

Epoch 00030: val_acc improved from 0.82840 to 0.85670, saving model to weights-improvement-30-0.86.hdf5
Epoch 31/35
36864/50000 [=====================>........] - ETA: 2:33 - loss: 0.1856 - acc: 0.9347

50000/50000 [==============================] - 630s 13ms/step - loss: 0.1871 - acc: 0.9339 - val_loss: 0.5510 - val_acc: 0.8666

Epoch 00031: val_acc improved from 0.85670 to 0.86660, saving model to weights-improvement-31-0.87.hdf5
Epoch 32/35
35072/50000 [====================>.........] - ETA: 2:54 - loss: 0.1782 - acc: 0.9365

50000/50000 [==============================] - 629s 13ms/step - loss: 0.1789 - acc: 0.9367 - val_loss: 0.6797 - val_acc: 0.8539

Epoch 00032: val_acc did not improve from 0.86660
Epoch 33/35
41600/50000 [=======================>......] - ETA: 1:38 - loss: 0.1721 - acc: 0.9388

50000/50000 [==============================] - 629s 13ms/step - loss: 0.1748 - acc: 0.9379 - val_loss: 0.8459 - val_acc: 0.8297

Epoch 00033: val_acc did not improve from 0.86660
Epoch 34/35
43520/50000 [=========================>....] - ETA: 1:15 - loss: 0.1651 - acc: 0.9414

50000/50000 [==============================] - 628s 13ms/step - loss: 0.1655 - acc: 0.9412 - val_loss: 0.7229 - val_acc: 0.8378

Epoch 00034: val_acc did not improve from 0.86660
Epoch 35/35
44160/50000 [=========================>....] - ETA: 1:08 - loss: 0.1657 - acc: 0.9410

50000/50000 [==============================] - 628s 13ms/step - loss: 0.1668 - acc: 0.9409 - val_loss: 0.7003 - val_acc: 0.8486

Epoch 00035: val_acc did not improve from 0.86660
10000/10000 [==============================] - 52s 5ms/step
Test loss: 0.7002615824878216
Test accuracy: 0.8486
Saved model to disk
attempt_1			  weights-improvement-26-0.83.hdf5
datalab				  weights-improvement-30-0.85.hdf5
DNST_model.h5			  weights-improvement-30-0.86.hdf5
drive				  weights-improvement-31-0.87.hdf5
weights-improvement-25-0.84.hdf5  weights-improvement-33-0.86.hdf5


In [24]:
#35-40
epoch=35
e = 40
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

Train on 50000 samples, validate on 10000 samples
Epoch 36/40
50000/50000 [==============================] - 626s 13ms/step - loss: 0.1552 - acc: 0.9450 - val_loss: 1.4790 - val_acc: 0.7479

Epoch 00036: val_acc did not improve from 0.86660
Epoch 37/40
15872/50000 [========>.....................] - ETA: 6:37 - loss: 0.1442 - acc: 0.9485

50000/50000 [==============================] - 627s 13ms/step - loss: 0.1526 - acc: 0.9460 - val_loss: 0.7985 - val_acc: 0.8401

Epoch 00037: val_acc did not improve from 0.86660
Epoch 38/40
35712/50000 [====================>.........] - ETA: 2:45 - loss: 0.1492 - acc: 0.9461

50000/50000 [==============================] - 627s 13ms/step - loss: 0.1528 - acc: 0.9455 - val_loss: 0.7576 - val_acc: 0.8439

Epoch 00038: val_acc did not improve from 0.86660
Epoch 39/40
41728/50000 [========================>.....] - ETA: 1:36 - loss: 0.1409 - acc: 0.9493

50000/50000 [==============================] - 627s 13ms/step - loss: 0.1442 - acc: 0.9480 - val_loss: 0.6216 - val_acc: 0.8688

Epoch 00039: val_acc improved from 0.86660 to 0.86880, saving model to weights-improvement-39-0.87.hdf5
Epoch 40/40
36352/50000 [====================>.........] - ETA: 2:38 - loss: 0.1439 - acc: 0.9485

50000/50000 [==============================] - 626s 13ms/step - loss: 0.1434 - acc: 0.9487 - val_loss: 0.7147 - val_acc: 0.8562

Epoch 00040: val_acc did not improve from 0.86880
10000/10000 [==============================] - 52s 5ms/step
Test loss: 0.7147344080418349
Test accuracy: 0.8562
Saved model to disk
attempt_1			  weights-improvement-30-0.85.hdf5
datalab				  weights-improvement-30-0.86.hdf5
DNST_model.h5			  weights-improvement-31-0.87.hdf5
drive				  weights-improvement-33-0.86.hdf5
weights-improvement-25-0.84.hdf5  weights-improvement-39-0.87.hdf5
weights-improvement-26-0.83.hdf5


In [58]:
model.load_weights('weights-improvement-39-0.87.hdf5')

#40-50
epoch=40
e = 50
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=e,
                    verbose=1,
                    initial_epoch=epoch,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

!ls
!cp *.hdf5 drive/EIP/
!cp *.h5 drive/EIP

Train on 50000 samples, validate on 10000 samples
Epoch 41/50
50000/50000 [==============================] - 662s 13ms/step - loss: 0.1467 - acc: 0.9469 - val_loss: 0.7625 - val_acc: 0.8430

Epoch 00041: val_acc improved from -inf to 0.84300, saving model to weights-improvement-41-0.84.hdf5
Epoch 42/50
13696/50000 [=======>......................] - ETA: 7:15 - loss: 0.1226 - acc: 0.9571

50000/50000 [==============================] - 645s 13ms/step - loss: 0.1360 - acc: 0.9516 - val_loss: 0.6692 - val_acc: 0.8596

Epoch 00042: val_acc improved from 0.84300 to 0.85960, saving model to weights-improvement-42-0.86.hdf5
Epoch 43/50
29184/50000 [================>.............] - ETA: 4:08 - loss: 0.1322 - acc: 0.9531

50000/50000 [==============================] - 644s 13ms/step - loss: 0.1345 - acc: 0.9520 - val_loss: 0.8009 - val_acc: 0.8440

Epoch 00043: val_acc did not improve from 0.85960
Epoch 44/50
39808/50000 [======================>.......] - ETA: 2:02 - loss: 0.1273 - acc: 0.9541

50000/50000 [==============================] - 647s 13ms/step - loss: 0.1284 - acc: 0.9537 - val_loss: 0.8075 - val_acc: 0.8421

Epoch 00044: val_acc did not improve from 0.85960
Epoch 45/50
43008/50000 [========================>.....] - ETA: 1:23 - loss: 0.1268 - acc: 0.9547

50000/50000 [==============================] - 644s 13ms/step - loss: 0.1269 - acc: 0.9548 - val_loss: 0.7416 - val_acc: 0.8527

Epoch 00045: val_acc did not improve from 0.85960
Epoch 46/50
44032/50000 [=========================>....] - ETA: 1:11 - loss: 0.1193 - acc: 0.9581

50000/50000 [==============================] - 642s 13ms/step - loss: 0.1209 - acc: 0.9574 - val_loss: 0.6001 - val_acc: 0.8741

Epoch 00046: val_acc improved from 0.85960 to 0.87410, saving model to weights-improvement-46-0.87.hdf5
Epoch 47/50
36992/50000 [=====================>........] - ETA: 2:34 - loss: 0.1159 - acc: 0.9598

50000/50000 [==============================] - 643s 13ms/step - loss: 0.1178 - acc: 0.9590 - val_loss: 0.7108 - val_acc: 0.8650

Epoch 00047: val_acc did not improve from 0.87410
Epoch 48/50
42112/50000 [========================>.....] - ETA: 1:34 - loss: 0.1154 - acc: 0.9590

50000/50000 [==============================] - 643s 13ms/step - loss: 0.1165 - acc: 0.9588 - val_loss: 0.5952 - val_acc: 0.8740

Epoch 00048: val_acc did not improve from 0.87410
Epoch 49/50
43776/50000 [=========================>....] - ETA: 1:14 - loss: 0.1093 - acc: 0.9619

50000/50000 [==============================] - 643s 13ms/step - loss: 0.1104 - acc: 0.9617 - val_loss: 0.7101 - val_acc: 0.8667

Epoch 00049: val_acc did not improve from 0.87410
Epoch 50/50
44288/50000 [=========================>....] - ETA: 1:08 - loss: 0.1134 - acc: 0.9588

50000/50000 [==============================] - 643s 13ms/step - loss: 0.1138 - acc: 0.9588 - val_loss: 0.6850 - val_acc: 0.8716

Epoch 00050: val_acc did not improve from 0.87410
10000/10000 [==============================] - 54s 5ms/step
Test loss: 0.6850166329085827
Test accuracy: 0.8716
Saved model to disk
attempt_1			  weights-improvement-33-0.86.hdf5
datalab				  weights-improvement-39-0.87.hdf5
DNST_model.h5			  weights-improvement-41-0.82.hdf5
drive				  weights-improvement-41-0.84.hdf5
weights-improvement-25-0.84.hdf5  weights-improvement-42-0.86.hdf5
weights-improvement-26-0.83.hdf5  weights-improvement-42-0.87.hdf5
weights-improvement-30-0.85.hdf5  weights-improvement-44-0.88.hdf5
weights-improvement-30-0.86.hdf5  weights-improvement-46-0.87.hdf5
weights-improvement-31-0.87.hdf5


In [33]:
model.load_weights('weights-improvement-44-0.88.hdf5')
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights("DNST_model.h5")
print("Saved model to disk")

10000/10000 [==============================] - 56s 6ms/step
Test loss: 0.6380089717403055
Test accuracy: 0.875
Saved model to disk
